## 使用Python制作神经网络

---

#### 介绍

我们将使用我们刚刚学习的 Python 知识制作神经网络。我们会从简单的开始，逐步建立 Python 程序。 先把整个神经网络需要的函数定义下来，搭好神经网络的框架。然后依次从神经网络的初始化，查询和训练入手。一步一步实现神经网络的所有过程。

#### 知识点

- 神经网络类的框架
- 神经的初始化
- 神经网络的查询
- 神经网络的训练

---

### 框架代码

让我们先来勾勒神经网络类的大概样子。它应该至少有 3 个函数：

- 初始化函数：设定输入层节点、隐藏层节点和输出层节点的数量。 
- 训练：学习给定的训练集样本后，优化权重。 
- 查询：给定输入，从输出节点得到答案（也就是所谓的测试）。 

目前，这些函数的实现还未定义，我们这里只是将整个代码的架构，搭建出来。代码如下：

In [ ]:
# 神经网络类 的定义
class neuralNetwork :

    # 初始化这个神经网络
    def __init__() :
        pass

    # 训练这个神经网络
    def train() :
        pass

    # 查询神经网络：给定输入，返回相对应的输出
    def query() :
        pass

这个开局不错。事实上，这是一个坚实的框架。在这个框架上，你可以充实神经网络工作的详细细节了。

### 初始化神经网络

从初始化网络开始。我们需要设置输入层节点、隐藏层节点和输出层节点的数量。这些节点数量定义了神经网络的形状和尺寸。为了能够多次使用，这里我们通过参数对这些值进行传递。这样我们就可以使用同一个类创建任何规模的神经网络了。

同时也请不要忘了学习率。当创建新的神经网络时，这也是待设置的有用参数。让我们看看 __init __() 函数是什么样子的：

In [ ]:
class neuralNetwork:
    # 初始化这个神经网络
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # 设置而输入层，隐藏层，输出层 的节点个数
        self.inputnodes = inputnodes
        self.hiddennodes = hiddennodes
        self.outputnodes = outputnodes

        # 设置学习率（即步长）
        self.learningrate = learningrate
        pass

让我们使用所定义的神经网络类，尝试创建每层的节点个数为 3、学习率为 0.5 的小型神经网络对象。 

In [ ]:
#  输入层，隐藏层，输出层 的节点个数为3
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# 学习率为 0.5
learning_rate = 0.5

# 传入参数，创建一个神经网络事例
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
print("该网络输入层的节点个数为：",n.inputnodes)
print("该网络隐藏层的节点个数为：",n.hiddennodes)
print("该网络输出层的节点个数为：",n.outputnodes)
print("该网络的学习率为：",n.learningrate)

上述代码创建了一个对象，并且我们利用 `print` 函数，查看了对象 n 中的每个网络层的节点个数，以及学习率。证明我们初始化函数编写完成。

那么，下一步该做些什么呢？接下来我们需要通过初始化的个数，来切实的创建具体的节点和链接权重。

#### 权重：网络的核心

网络中最重要的部分就是链接权重，我们使用这些权重来计算前馈信号以及反向传播误差。并且在试图改进网络时优化链接权重本身。

之前，我们从章节里学到了，如何使用矩阵简明地表示权重。

因此，我们可以创建：

- 在输入层与隐藏层之间的链接权重矩阵 $W_{input\_hidden}$ ，矩阵大小为 hidden_nodes $\times$ input_nodes 。 
- 在隐藏层和输出层之间的链接权重矩阵 $W_{hidden\_output}$，矩阵大小为 hidden_nodes $\times$ output_nodes 。

请前谨记上面的规则，至于说为什么第一个矩阵的大小是 hidden_nodes $\times$  input_nodes ，而不是 input_nodes $\times$  hidden_node，后面我们会具体进行阐述。

从之前的实验中，我们了解到，链接权重的初始值应该较小，并且是随机的。下面的 Numpy 函数生成了一个数组，数组中元素为 0～1 的随机值，数组的大小为 $rows \times columns$。 

In [ ]:
import numpy
numpy.random.rand(3,3)#随机初始化，一个 3 行 3 列的矩阵

下图显示了这个函数可以生成 $3\times3$ 的 Numpy 数组。数组中的每个值都是 0～1 的随机值。

其实我们可以做得更好。因为权重可以为正数也可以为负数。而上面的 `random` 生成的却是 0~1 的随机值。那么我们有没有办法让它生成 -0.5 ~ +0.5 的随机值呢？当然，我们只需要将随机得到的值，减去 0.5 即可。如下代码所示：

In [ ]:
numpy.random.rand(3,3)-0.5 #随机初始化了 一个 3 行 3 列，值为 -0.5~0.5 的矩阵

我们已经准备好了，在 Python 程序中创建初始权重矩阵。权重是神经网络的固有部分，与神经网络共存亡，不会随着函数调用的结束而消失。

下面的代码创建了两个链接的权重矩阵。并使用 self.inodes、self.hnodes 和 self.onodes 为两个链接权重矩阵设置合适的大小。

In [ ]:
#三层神经网络，中间夹了两层权重矩阵
#第一层，输入层与隐藏层的连接矩阵，矩阵大小为( hnodes × inodes )
#且矩阵中的每一个值都是-0.5 ~ 0.5 的随机值
wih = (numpy.random.rand(hidden_nodes, input_nodes) - 0.5)
#第二层权重矩阵，同理
who = (numpy.random.rand(output_nodes, hidden_nodes) - 0.5) 
wih,who

当然除了随机初始化权重之外，我们也是可以利用一些较为复杂的权重初始方法。

#### 可选项：较复杂的权重

我们可以选择这种简单却流行的优化初始权重的方式。

根据之前实验章节对初始化权重的讨论，你会发现，有些人更喜欢稍微复杂的方法来创建初始随机权重。他们使用正态概率分布采样权重。在这个正态分布中，所使用到的平均值为 0，标准差为 $\frac{1}{\sqrt{传入链接数目}}$ 。

在 Numpy 程序库的帮助下，这是很容易实现的。numpy.random.normal() 函数可以帮助我们以正态分布的方式进行采样。

其代码如下所示： 

In [ ]:
# numpy.random.normal 的参数分别为：
# 正态分布所需的平均值，
# 正态分布所需的标准差，
# 需要初始化的矩阵大小（ rows × columns ）。
wih = numpy.random.normal(0.0, pow(hidden_nodes, -0.5),
                          (hidden_nodes, input_nodes))
who = numpy.random.normal(0.0, pow(output_nodes, -0.5),
                          (output_nodes, hidden_nodes))
wih,who

其中 `pow（x,y）` 表示 $x^y$，因此 pow( hidden_nodes , -0.5 ) 表示 $\frac{1}{\sqrt{hidden_nodes}}$ 。

### 查询网络

接下来，顺理成章，我们现在应该编写训练神经网络的代码，填写当前空的 train() 函数。但是，还是等一下再写 train() 函数，让我们先编写简单的 query() 函数吧。

query() 函数接受神经网络的输入，返回网络的输出。这个功能非常简单，我们中需要传递来自输入层节点的输入信号，通过隐藏层，最后从输出层输出。你还要记住，当信号馈送至给定的隐藏层节点或输出层节点时，我们使用链接权重调节信号，还应用 S 激活函数来抑制信号。

下式显示了输入层和隐藏层之间的链接权重矩阵如何与输入矩阵相乘，给出隐藏层节点的输入信号。

$$X_{hidden}=W_{input\_hidden}\cdot I$$

这样做的好处，不仅是更容易书写，而且如 Python 这样的编程语言也可以理解矩阵，由于这些编程语言认识到所有这些基础计算之间的相似之处，它们可以非常有效率地完成所有实际工作。

以下代码应用了 Numpy 代码库，将链接权重矩阵 $W_{input\_hidden}$ 点乘输入矩阵 $I$。

In [ ]:
#现在输入一条数据 input ，矩阵大小为 ( 3 乘 1 )
input = numpy.random.randn(3,1) # ( 3 乘 1 )
hidden_inputs = numpy.dot(wih, input) # wih 的大小为 2
hidden_inputs

这一段简单的 Python 完成了所有的工作，将所有的输入与所有正确的链接权重组合，生成了组合调节后的信号矩阵，传输给每个隐藏层节点。

这种力量，优雅而且简洁。这也就是为什么之前，我们会花大功夫来讲解矩阵的原因。

为了获得从隐藏层节点处出现的信号，我们简单地将 $S$ 抑制函数应用到每一个出现的信号上：

$$ O_{hidden} = sigmoid(X_{hidden})$$

如果在某个现成的 Python 库中，已经定义了这个 S 函数，那么这种操作就变得非常容易了。果不其然！SciPy Python 库有一组特殊的函数，在这组函数中，S 函数称为 expit() 。不要问我为什么 S 函数有这样一个愚蠢的名字。可以像导入 Numpy 程序库一样，导入 Scipy 函数库：

In [ ]:
# 导入 scipy.special 方便后面使用 sigmoid 函数 expit() 
import scipy.special

我们在初始化函数中声明这个激活函数，方便在神经网络类的其它函数中调用。

In [ ]:
activation_function = lambda x: scipy.special.expit(x) 
activation_function

这是什么代码？它看起来不像我们以前见过的任何代码。lambda 是什么？请静下心来，其实这并不可怕。这里所做的一切就是创建一个函数，就像创建其他函数一样，不过我们使用了较短的方式将这个函数写出来了。我们不使用通常的 def() 来定义函数，我们使用神奇的 lambda 来创建函数，方便又快捷。

这个函数接受了 x，返回 scipy.special.expit（x） ，这就是 S 函数。使用 lambda 创建的函数是没有名字的，经验丰富的程序员喜欢称它们为匿名函数，但是这里分配给它一个名字叫做 activation_function() 。所有这些事情意味着，无论何时任何人需要使用激活函数，那么他所需做的就是调用 activation_function() 。

回到手上的任务，我们要将激活函数应用到准备进入隐藏层节点的信号。其代码与下面的代码一样简单：

In [ ]:
# 来自隐藏层的组合信号
hidden_outputs = activation_function(hidden_inputs)
hidden_outputs

也就是说，隐藏层节点的输出信号在名为 hidden_outputs 的矩阵中。

至此，我们已经将信号传达到了中间隐藏层，那么信号如何到达最终输出层呢？其实，在隐藏层节点和最终输出层节点之间并没有什么本质的区别。因此过程也是一样的。这意味着代码也非常相似。

看看下面的代码，这些代码总结了我们如何计算隐藏层信号和输出层信号。

In [ ]:
# 计算进入隐藏层的信号
hidden_inputs = numpy.dot(wih, input)
# 对进入隐藏层的信号进行激活处理（S 函数处理）
hidden_outputs = activation_function(hidden_inputs)

# 计算进入最后输出层的信号
final_inputs = numpy.dot(who, hidden_outputs)
# 对进入输出层层的信号进行激活处理（S 函数处理）
final_outputs = activation_function(final_inputs)
print("input：",final_inputs)
print("output：",final_outputs)

如果删除注释，那么只有四行粗体显示的代码进行了所需的计算。其中前两行为隐藏层，后两行为最终输出层。

### 迄今为止的代码

让我们喘一口气，停下来，把上面讲的代码全部整合进神经网络中。

In [ ]:
# 导入该工具包的目的是为了实现矩阵相关的操作
import numpy
import scipy.special


class neuralNetwork:

    # 初始化神经网络
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # 设置输入层，隐藏层，输出层节点的个数
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # 利用正态分布，随机初始化两层的权重矩阵
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5),
                                       (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5),
                                       (self.onodes, self.hnodes))
        # 初始化学习率
        self.lr = learningrate

        # 利用 special.expit(x) 函数，定义本神经网络所需要的 S 激活函数
        self.activation_function = lambda x: scipy.special.expit(x)
        pass

    # 训练神经网络函数：还未书写
    def train():
        pass

    
    # 查询神经网络
    def query(self, inputs_list):
        # 将输入的 lists 转为二维矩阵。
        # 转置一下的目的，是让一列成为一条数据
        inputs = numpy.array(inputs_list, ndmin=2).T

        # 计算进入隐藏层的信号
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 对进入隐藏层的信号进行激活处理（S 函数处理）
        hidden_outputs = self.activation_function(hidden_inputs)

        # 计算进入最后输出层的信号
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 对进入输出层层的信号进行激活处理（S 函数处理）
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

值得一提的是，query() 函数只需要 inputs_list 。它不需要任何其他输入。

在继续编写 train() 函数并使用样本训练网络之前，让我们测试目前得到的所有的代码。我们创建一个小网络，使用一些随机输入查询网络，看看网络如何工作。显而易见，这样做不会有任何实际意义，只是为了使用刚刚创建的函数。

下面代码，创建了这个小型的网络。其中，在输入层、隐藏层和输出层中，每层有 3 个节点。

In [ ]:
#创建一个输入层，隐藏层，输出层的节点数都为 3 ，且学习率为 0.3 的神经网络实例 n
input_nodes = 3
hidden_nodes = 3
output_nodes = 3
learning_rate = 0.3
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)
n

我们传入，随机信号（1.0，0.5，-1.5）查询神经网络 n 的输出（即预测结果）：

In [ ]:
n.query([1.0,0.5,-1.5])

输入是一个列表，Python 语言将列表写在方括号内。输出也是数字列表。由于还没有训练网络，这个输出没有实际意义，但是我们还是很高兴代码没有出错。

### 训练网络

现在来解决这个稍微复杂的训练任务。训练任务分为两个部分：

- 第一部分，针对给定的训练样本计算输出。这与我们刚刚在 query() 函数上所做的没什么区别。 
- 第二部分，将计算得到的输出与所需输出对比，使用差值来指导网络权重的更新。

我们已经完成了第一部分，现在，先把这部分写出来：

In [ ]:
#这里我们随机假设一个训练集
inputs_list = [1.0,0.5,-1.5] #训练集中的一条数据
targets_list = [0.3,1.3,0.7]   #该条数据对应的真实输出
targets = numpy.array(targets_list, ndmin=2).T

# 下面五行代码和 query（） 中计算最后输出的代码完全一致
inputs = numpy.array(inputs_list, ndmin=2).T
hidden_inputs = numpy.dot(wih, inputs)
hidden_outputs = activation_function(hidden_inputs)
final_inputs = numpy.dot(who, hidden_outputs)
final_outputs = activation_function(final_inputs)

我们使用完全相同的方式从输入层前馈信号到最终输出层，所以这段代码与在 query() 函数中的几乎完全一样。

由于需要使用每条数据的真实标签来训练网络，因此唯一的区别是，这部分代码中有一个额外的参数。即在函数的名称中定义的  targets_list 。

现在，我们越来越接近神经网络工作的核心，即基于所计算输出与目标输出之间的误差，改进权重。

让我们按照轻柔可控的步骤，进行这种操作。

首先需要计算误差，这个值等于训练样本所提供的真实输出值与计算得到的输出值之差。这个差也就是将矩阵 targets 和矩阵   final_outputs 中每个对应元素相减得到的。Python 代码非常简单，这再次优雅地显示了矩阵的力量。

In [ ]:
# 计算误差 =  (target - actual) 
output_errors = targets - final_outputs
output_errors  # 将误差值输出

我们可以计算出隐含层节点的反向传播误差。请回忆一下，如何根据所连接的权重分割误差，为每个隐藏层节点重组这些误差。对于这个计算过程，我们得到了其矩阵形式：

$$errors_{hidden} = weights^T_{hidden\_output} \cdot errors_{output}$$

由于 Python 有能力使用 Numpy 进行点乘，因此，这段代码再简单不过了：

In [ ]:

# 隐藏层的误差 是通过输出层的误差根据不同的权重，反向传播得到的。
hidden_errors = numpy.dot(who.T, output_errors)
print("隐藏层误差：",hidden_errors)

这样，我们就拥有了所需要的一切，可以优化各个层之间的权重了。

对于在隐蔽层和最终层之间的权重，我们使用 output_errors 进行优化。对于输入层和隐藏层之间的权重，我们使用刚才计算得到的 hidden_errors 进行优化。

先前，我们得到了用于更新节点 $j$ 与其下一层节点 $k$ 之间链接权重的矩阵形式的表达式：

<img width="600px" src="https://doc.shiyanlou.com/courses/uid1166617-20191220-1576809035484/wm">

$α$ 是学习率，$Sigmoid$ 是先前看到的激活函数。请记住，乘法是正常的对应的元素相乘，• 点乘是矩阵的点积。

最后一点要注意，来自上一层的输出矩阵被转置了。实际上，这意味着输出矩阵的列变成了行。

下面我们利用 Python 代码实现上面的这个公式，并且最终完成隐藏层与输出层之间的权重的更新。

In [ ]:
print("原 h-o 的权重：", who)
# 更新隐藏层和输出层之间的权重。
who += learning_rate * numpy.dot((   
    output_errors * final_outputs * (1.0 - final_outputs)), #表示 E_k * sigmod * （1 - sigmod）
    numpy.transpose(hidden_outputs)) # 表示最后一项 OjT
print("更新后 h - o 的权重：", who)

等号右边的公式，是在计算需要增加的权重值： $△w_{j，k}$ 。将增加量计算出来之后，再用原来的权重值，加上权重的增加量，最终得到更新后的权重值。

上面代码中，$x + = 3$ ，意思就是 $x$ 增加 $3$ 。也就是 $x = x + 3$ 的简短写法。

在输入层和隐藏层之间权重的代码也是类似的，其公式还是上面那个公式。我们只是利用对称性，重写代码，更换名字，这样它们指的就是神经网络的前一层了。代码如下：

In [ ]:
print("原  i - h 的权重：", wih)
# 更新输入层与隐藏层之间的权重
wih += learning_rate * numpy.dot(( hidden_errors * hidden_outputs *
                                  (1.0 - hidden_outputs) ), numpy.transpose(inputs) )
print("更新后i - h 的权重：", who)

至此，训练的代码编写完毕，所有的权重都进行了一次更新。我们只需要不断的重复上面的训练操作，不断的更新，就能找到较好的权重值。

我们先前进行了海量的计算、并努力获得了矩阵方法、通过梯度下降法最小化网络误差……所有的这些都变成了以上简短而简洁的代码，这真是难以置信！在某种意义上，这是 Python 力量的表现。但是，实际上，这也是我们努力工作并简化公式的必然结果。最后我们把上面的代码，写入神经网络结构的 train() 函数函数中。

In [ ]:
# 神经网络类的定义
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):  
           pass

    
    # 神经网络类的训练函数
    #除了 self 外还要接受两个陈述
    #inputs_list: 训练数据的输入集合
    #targets_list: 训练数据的输出集合
    def train(self, inputs_list, targets_list):
        # 将输入集合转换为 Numpy 的 2 维矩阵
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # 计算进入隐藏层的信号，通过输入信号与权重的组合得到
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 对组合信号进行  S 函数抑制，得到隐藏层的输出信号
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # 计算最终进入输出层的信号
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 对组合信号进行  S 函数抑制，得到输出层的输出信号
        final_outputs = self.activation_function(final_inputs)
        
        # 计算输出层需要的损失误差
        output_errors = targets - final_outputs
        # 通过损失误差的反向传播，得到更新进入隐藏层的权重所需要的损失误差
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # 更新隐藏层到输出层之间的权重
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # 更新输入层到隐藏层之间的权重
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    def query(self, inputs_list):
           pass

这些代码可用于创建、训练和查询 3 层神经网络，进行几乎任何任务，这么看来，代码不算太多。 

### 实验总结

至此，我们完成了一个神经网络类的所有功能函数。下一个实验，我们将会利用上，我们所写的神经网络完成一个，真正意义上的项目：手写数字的识别。

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>